<a href="https://colab.research.google.com/github/beercafeguy/PyMLLab/blob/master/grokking_dl/grokking_dl_chapter_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

from keras.datasets import mnist

In [2]:
(X_train, y_train) , (X_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [3]:
!ls -ltr

total 4
drwxr-xr-x 1 root root 4096 Feb 13 14:23 sample_data


In [6]:
images, labels = (X_train[0:1000].reshape(1000,28*28) / 255, y_train[0:1000])

In [7]:
one_hot_labels = np.zeros((len(labels),10))

In [8]:
for i,l in enumerate(labels): one_hot_labels[i][l] = 1
labels = one_hot_labels

In [9]:
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
images

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
test_images = X_test.reshape(len(X_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
  test_labels[i][l] = 1

np.random.seed(2024)

In [17]:
relu = lambda x : (x > 0) * x
relu2deriv = lambda x: x>=0

In [13]:
alpha, iterations, hidden_size, pixels_per_image, num_labels = (0.005, 350, 40, 784, 10)

In [14]:
weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

In [20]:
import sys
for j in range(iterations):
  error, correct_cnt = (0.0, 0)
  for i in range(len(images)):
    layer_0 = images[i:i+1]

    dropout_mask = np.random.randint(2,size=layer_1.shape)
    layer_1 = relu(np.dot(layer_0,weights_0_1))
    layer_1 *= dropout_mask * 2

    layer_2 = np.dot(layer_1,weights_1_2)
    error += np.sum((labels[i:i+1] - layer_2) ** 2)
    correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i:i+1]))
    layer_2_delta = (labels[i:i+1] - layer_2)
    layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
    layer_1_delta *= dropout_mask
    weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
    weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
    sys.stdout.write("\r"+
    " I:"+str(j)+ \
    " Error:" + str(error/float(len(images)))[0:5] + " Correct:" + str(correct_cnt/float(len(images))))

 I:349 Error:0.346 Correct:0.87

In [21]:
#testing on actual data

if(j % 10 == 0 or j == iterations-1):
  error, correct_cnt = (0.0, 0)
  for i in range(len(test_images)):
    layer_0 = test_images[i:i+1]
    layer_1 = relu(np.dot(layer_0,weights_0_1))
    layer_2 = np.dot(layer_1,weights_1_2)
    error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
    correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
  sys.stdout.write(" Test-Err:" + str(error/float(len(test_images)))[0:5] + " Test-Acc:" + str(correct_cnt/float(len(test_images))))
  print()

 Test-Err:0.432 Test-Acc:0.7948


In [ ]:
# train the same network with mini batch gradient descent

relu = lambda x : (x > 0) * x
relu2deriv = lambda x: x>=0


batch_size = 100
alpha, iterations = (0.001, 300)
pixels_per_image, num_labels, hidden_size = (784, 10, 100)

weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1


for j in range(iterations):
  error, correct_cnt = (0.0, 0)
  for i in range(int(len(images) / batch_size)):
    batch_start, batch_end = ((i * batch_size),((i+1)*batch_size))
    layer_0 = images[batch_start:batch_end]
    layer_1 = relu(np.dot(layer_0,weights_0_1))

    dropout_mask = np.random.randint(2,size=layer_1.shape)
    layer_1 *= dropout_mask * 2
    layer_2 = np.dot(layer_1,weights_1_2)

    error += np.sum((labels[batch_start:batch_end] - layer_2) ** 2)
    for k in range(batch_size):
      correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(labels[batch_start+k:batch_start+k+1]))
      layer_2_delta = (labels[batch_start:batch_end]-layer_2) /batch_size
      layer_1_delta = layer_2_delta.dot(weights_1_2.T)* relu2deriv(layer_1)
      layer_1_delta *= dropout_mask
      weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
      weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
    sys.stdout.write("\r"+
    " I:"+str(j)+ \
    " Error:" + str(error/float(len(images)))[0:5] + " Correct:" + str(correct_cnt/float(len(images))))
  if(j%10 == 0):
    test_error = 0.0
    test_correct_cnt = 0
    for i in range(len(test_images)):
      layer_0 = test_images[i:i+1]
      layer_1 = relu(np.dot(layer_0,weights_0_1))
      layer_2 = np.dot(layer_1, weights_1_2)


 I:158 Error:0.154 Correct:0.343